Importar las librerias que se necesitan

In [1]:
import pandas as pd
import os
import glob
import yfinance as yf
from datetime import datetime
import math
import numpy as np
import requests
from bs4 import BeautifulSoup

Descargar los datos necesarios desde Yahoo Finance (se puede automatizar con airflow)

In [2]:
"""

## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos, 87 Minutos 
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock
for archivo_csv in archivos_csv:
    # Obtener el símbolo del stock desde el nombre del archivo
    nombre_archivo = os.path.basename(archivo_csv)
    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock
    
    # Descargar los datos desde Yahoo Finance para los últimos 10 años
    data = yf.download(simbolo, start='2013-01-01')
    
    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)
    data = data[['Open', 'Close', 'Adj Close', 'Volume']]
    
    # Guardar los datos en un nuevo archivo CSV
    nuevo_nombre_archivo = f'{simbolo}.csv'
    ruta_nuevo_archivo = os.path.join(directorio, nuevo_nombre_archivo)
    data.to_csv(ruta_nuevo_archivo)

"""

"\n\n## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos, 87 Minutos \n# Ruta al directorio donde se encuentran los archivos CSV\ndirectorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'\n\n# Patrón para buscar archivos CSV\npatron = '*.csv'\n\n# Obtener la ruta de directorio de cada uno\narchivos_csv = glob.glob(os.path.join(directorio, patron))\n\n# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock\nfor archivo_csv in archivos_csv:\n    # Obtener el símbolo del stock desde el nombre del archivo\n    nombre_archivo = os.path.basename(archivo_csv)\n    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock\n    \n    # Descargar los datos desde Yahoo Finance para los últimos 10 años\n    data = yf.download(simbolo, start='2013-01-01')\n    \n    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)\n    data = data[['Op

Se extraen los nombres de las acciones de NASDAQ y NYSE

In [2]:
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Lista para almacenar los nombres de los archivos CSV
nombres_archivos_csv = []

# Extraer el nombre de cada archivo CSV
for archivo in archivos_csv:
    nombres_archivos_csv.append(os.path.basename(archivo))

Se crea un DataFrame por cada Stock

In [3]:
# Lista para almacenar los DataFrames de los archivos CSV
dataframes_csv = []
# Leer cada archivo CSV y almacenar el DataFrame correspondiente
for i in range(len(nombres_archivos_csv)):
    ruta_archivo = directorio + '/' + nombres_archivos_csv[i]  # Ruta completa al archivo
    df = pd.read_csv(ruta_archivo)
    dataframes_csv.append(df)

# Crea el DF por accion
for i in range(len(nombres_archivos_csv)):
    # Extraer un nombre de la lista
    nombre_variable = nombres_archivos_csv[i].replace('.csv','')
    # Asignar un valor a la variable con el nombre extraído
    globals()[nombre_variable] = dataframes_csv[i]

Se crea una lista con el nombre de cada stock (No la ruta del archivo)

In [4]:
# Extraer sólo el nombre del stock
for i in range(len(nombres_archivos_csv)):
    nombres_archivos_csv[i] = nombres_archivos_csv[i].replace('.csv','')

Se empiezan a eliminar los Stocks que no se pueden utilizar 

Se eliminan los df que no tengan datos, que no tengan histórico desde el primer día hábil del 2013, que no estén cotizando para el día hábil más cercano de cuando descargamos los datos, y que tengan información continua (que no falte ninguna cotización)

In [5]:
archivos_inutiles = []
length = len(globals()[nombres_archivos_csv[0]]['Date'])
ult_fecha = A['Date'][len(A['Date'])-1]

# Iterar sobre los nombres de archivos CSV
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    fecha = dataframe['Date']
    
    if len(fecha) != length or fecha[0] != '2013-01-02'  or fecha[len(fecha) - 1] != ult_fecha:
        archivos_inutiles.append(nombre_archivo)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for nombre_archivo in archivos_inutiles:
    nombre_dataframe = nombre_archivo.replace('.csv', '')
    del globals()[nombre_dataframe]
    nombres_archivos_csv.remove(nombre_archivo)

Calcular Desviación Estandard y rango de precios 

In [6]:
"""# ESTÁ EN EL ARCHIVO DE METRICS
metrics = pd.DataFrame()

metrics['Stock'] = nombres_archivos_csv
std = []

for i in range(len(metrics['Stock'])):
    std.append(np.std(globals()[metrics['Stock'][i]]['Adj Close']))

metrics['Standard Deviation'] = std"""


In [7]:
"""# ESTÁ EN EL ARCHIVO DE METRICS
r_min = []
r_max = []
fecha = '2023-07-14'    ## Fecha de 'Ayer', del dia anterior para basarse ena estimación del precio de "hoy" (Siempre tiene que ser de días hábiles)
indice = globals()[metrics['Stock'][0]].loc[A['Date'] == fecha].index[0]    ## Como todos los archivos tienen la misma cantidad de datos, si en uno se encuentra la fecha en x 
                                                                            ## posicion entonces en esa misma estará para todos los demás df
for i in range(len(metrics['Stock'])):
    r_min.append(globals()[metrics['Stock'][i]]['Close'][indice] - metrics['Standard Deviation'][i])
    r_max.append(globals()[metrics['Stock'][i]]['Close'][indice] + metrics['Standard Deviation'][i])

metrics['Rango Mínimo Estimado'] = r_min
metrics['Rango Máximo Estimado'] = r_max"""

Calcular rendimientos diarios, mensuales y anuales con su respectivo promedio

In [6]:
## PARA CALCULAR LOS RENDIMIENTOS DIARIOS -> (Pf/Pi)-1

for j in range(len(nombres_archivos_csv)):
    nombre_archivo = nombres_archivos_csv[j]
    dataframe = globals()[nombre_archivo]
    dataframe['Rendimientos diarios (%)'] = np.nan
    adj_close = dataframe['Adj Close']
    rendimientos = [((adj_close[i] / adj_close[i-1]) - 1)*100 for i in range(1, len(adj_close))]
    dataframe['Rendimientos diarios (%)'][1:] = rendimientos

C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_14160\90114443.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_14160\90114443.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_14160\90114443.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [9]:
"""## CALCULAR PROMEDIO DE RENDIMIENTOS DIARIOS POR STOCK
# ESTÁ EN EL ARCHIVO DE METRICS
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(globals()[metrics['Stock'][i]]['Rendimientos diarios (%)']))
metrics['Rendimiento promedio diario (%)'] = p_rend"""

In [10]:
"""## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK
# ESTÁ EN EL ARCHIVO DE METRICS
# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_mensuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]

    # Antes de aplicar resample, convertimos la columna 'Date' en un índice de fecha
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    monthly_data = dataframe.resample('M', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    monthly_data['Rendimiento mensual (%)'] = (monthly_data['Adj Close']['last'] / monthly_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_mensuales[nombre_archivo] = monthly_data"""


In [11]:
"""## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK
# ESTÁ EN EL ARCHIVO DE METRICS
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_mensuales[metrics['Stock'][i]]['Rendimiento mensual (%)']))
metrics['Rendimiento promedio mensual (%)'] = p_rend"""

In [12]:
"""## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK
# ESTÁ EN EL ARCHIVO DE METRICS
# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_anuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    annual_data = dataframe.resample('A', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    annual_data['Rendimiento Anual (%)'] = (annual_data['Adj Close']['last'] / annual_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_anuales[nombre_archivo] = annual_data

# Ahora puedes acceder a los DataFrames individuales con sus nombres
# por ejemplo, dataframes_rendimientos_mensuales['NombreDelDataFrame']"""

In [13]:
"""## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK
# ESTÁ EN EL ARCHIVO DE METRICS
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_anuales[metrics['Stock'][i]]['Rendimiento Anual (%)']))
metrics['Rendimiento promedio anual (%)'] = p_rend"""

Calcular volatilidad utilizando los métodos Historical Volatility (HV) y Realized Volatility (RV) 

In [15]:
"""## METODO "Historical Volatility (HV)" 
    # HV = √VAR * √N
    # VAR = SD/N

var = []
HV = []
dias_negociados = len(A['Date'])
for i in range(len(metrics['Stock'])):
    var.append((metrics['Standard Deviation'][i])/(dias_negociados))
    HV.append(math.sqrt(var[i]) * math.sqrt(dias_negociados))
metrics['Historical Volatility'] = HV"""

In [17]:
"""## METODO "Realized Volatility (RV)" 
    # RV = √((∑SDR)/N)
    # SDR = (Daily return)^2
SDR = []  # Para agregar la suma de cada stock

for j in range(len(metrics['Stock'])):
    sdr = []  # Reiniciar la lista de SDR para cada stock
    for i in range(1, dias_negociados):
        sdr.append((globals()[metrics['Stock'][j]]['Rendimientos diarios (%)'][i]) ** 2)
    SDR.append(sum(sdr))

# Ahora calculamos la volatilidad (RV) para cada stock
metrics['Realized Volatility'] = [((sdr_sum / dias_negociados) ** 0.5) for sdr_sum in SDR]
"""

    Baja volatilidad: Menos del 10% de desviación estándar.
    Volatilidad moderada: Entre el 10% y el 20% de desviación estándar.
    Alta volatilidad: Más del 20% de desviación estándar.

### Calcular valor intrínseco 

    Primero hay que calcular:
        Terminal Value (TV) 
        Terminal Growth Rate (g)
        Retention Ratio ***

Calcular Retention Ratio

* NET INCOME

In [25]:
## WEB SCRAPING PARA EXTRAER EL NET INCOME

def get_net_income(ticket):
    url = f"https://www.marketwatch.com/investing/stock/{ticket}/financials/income/quarter"

    # Realizar una solicitud GET a la página web
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de respuesta 200)
    if response.status_code == 200:
        # Parsear el contenido HTML de la página con BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Buscar el elemento que contiene el net income
        net_income_element = soup.find("div", class_="cell__content", string="Net Income")

        if net_income_element:
            # Obtener todas las etiquetas div con clase "cell__content" dentro de la fila del net income
            net_income_values = net_income_element.find_parent("tr").find_all("div", class_="cell__content")

            # Acceder al penúltimo elemento de la lista para obtener el valor del net income
            net_income = net_income_values[2].text.strip() # Para el primer trimestre 
            # net_income = net_income_values[-2].text.strip() # Para el último trimestre
            return net_income
        else:
            return "Net Income no encontrado en la tabla de Income Statement."
    else:
        return "No se pudo acceder a la página web."



In [58]:
"""## ES MEJOR CORRERLO DE 500 EN 500 PORQUE SI SE CAE LA CONEXIÓN A LA RED ENTONCES NO TOMA NINGUN DATO
# net_income = []
for i in range(2000,len(metrics['Stock'])):
    net_income.append(get_net_income(metrics['Stock'][i]))"""

In [61]:
"""# Para verificar si todo está bien con todos los datos
no_access = 0
no_info = 0
for i in range(len(net_income)):
    if net_income[i] == 'No se pudo acceder a la página web.':
        no_access += 1
    if net_income[i] == 'Net Income no encontrado en la tabla de Income Statement.':
        no_info += 1
print('Sin acceso:',no_access,'/','Sin info:',no_info)"""

Sin acceso: 0 / Sin info: 0


Los datos se muestran con la M o B anexada, o con el () para mostrar que es un dato negativo.
// Data cleaning

In [64]:
"""for i in range(len(net_income)):
    net_income[i] = net_income[i].replace('-','0')
    net_income[i] = net_income[i].replace('(','-')
    net_income[i] = net_income[i].replace(')','')
    net_income[i] = net_income[i].replace('M','000000')
    net_income[i] = net_income[i].replace('B','0000000')
    net_income[i] = net_income[i].replace('K','000')
    net_income[i] = net_income[i].replace('T','000000000000')
    net_income[i] = net_income[i].replace('.','')
# metrics['Net Income'] = net_income"""

Hay stocks que no son empresas, son fondos, y en ellos no se debería invertir; algunas otras si son empresas pero están en proceso de quiebra o ya están liquidadas. 

In [66]:
"""archivos_inutiles = []
for i in range(len(metrics['Net Income'])):
    if metrics['Net Income'][i] == 'Net Income no encontrado en la tabla de Income Statement':
        archivos_inutiles.append(i)
    if metrics['Net Income'][i] == 'No se pudo acceder a la página web':
        archivos_inutiles.append(i)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for i in archivos_inutiles:
    del globals()[metrics['Stock'][i]]
    nombres_archivos_csv.remove(metrics['Stock'][i])

# Eliminar los stocks inútiles del DataFrame "Metrics"
metrics = metrics.drop(archivos_inutiles)"""

In [67]:
"""# Reiniciar los indices de metrics para evitar problemas futuros 
metrics.reset_index(drop=True, inplace=True)
metrics.index += 0"""

In [ ]:
"""# Convertir toda la fila en int
metrics['Net Income'] = pd.to_numeric(metrics['Net Income'])"""

* DIVIDENDOS

In [8]:
## WEB SCRAPING PARA EXTRAER LOS DIVIDENDOS

def get_cash_dividends_paid_total(ticket):
    url = f"https://www.marketwatch.com/investing/stock/{ticket}/financials/cash-flow/quarter"

    # Realizar una solicitud GET a la página web
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de respuesta 200)
    if response.status_code == 200:
        # Parsear el contenido HTML de la página con BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Buscar el elemento que contiene los "Cash Dividends Paid - Total"
        cash_dividends_element = soup.find("div", class_="cell__content", string="Cash Dividends Paid - Total")

        if not cash_dividends_element:
            url = f'https://www.marketwatch.com/investing/fund/{ticket}/financials/cash-flow/quarter'
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            cash_dividends_element = soup.find("div", class_="cell__content", string="Cash Dividends Paid - Total")

        if cash_dividends_element:
            # Obtener todas las etiquetas div con clase "cell__content" dentro de la fila de los "Cash Dividends Paid - Total"
            cash_dividend_values = cash_dividends_element.find_parent("tr").find_all("div", class_="cell__content")

            # Acceder al penúltimo elemento de la lista para obtener el valor de los "Cash Dividends Paid - Total"
            cash_dividends_paid_total = cash_dividend_values[-2].text.strip()
            return cash_dividends_paid_total
        else:
            return "Cash Dividends Paid - Total no encontrado en la tabla de Cash Flow."
    else:
        return "No se pudo acceder a la página web."


In [88]:
"""## ES MEJOR CORRERLO DE 300 EN 300 PORQUE SI SE CAE LA CONEXIÓN A LA RED ENTONCES NO TOMA NINGUN DATO
dividends = []
for i in range(0,len(metrics['Net Income'])):
    dividends.append(get_cash_dividends_paid_total(metrics['Stock'][i]))"""

In [91]:
"""no_access = 0
no_info = 0
for i in range(len(dividends)):
    if dividends[i] == 'No se pudo acceder a la página web.':
        no_access += 1
    if dividends[i] == 'Cash Dividends Paid - Total no encontrado en la tabla de Cash Flow.':
        no_info += 1
print('Sin acceso:',no_access,'/','Sin info:',no_info)"""

Sin acceso: 0 / Sin info: 72


Para algunos fondos o empresas (101) no estaba el dato de dividendos, ese hecho no hace que la empresa sea confiable

In [94]:
"""for i in range(len(dividends)):
    dividends[i] = dividends[i].replace('-','0')
    dividends[i] = dividends[i].replace('(','-')
    dividends[i] = dividends[i].replace(')','')
    dividends[i] = dividends[i].replace('M','000000')
    dividends[i] = dividends[i].replace('B','0000000')
    dividends[i] = dividends[i].replace('K','000')
    dividends[i] = dividends[i].replace('T','000000000000')
    dividends[i] = dividends[i].replace('.','')
metrics['Dividendos'] = dividends"""

In [106]:
"""archivos_inutiles = []
for i in range(len(metrics['Dividendos'])):
    if metrics['Dividendos'][i] == 'Cash Dividends Paid 0 000000000000otal no encontrado en la tabla de Cash Flow':
        archivos_inutiles.append(i)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for i in archivos_inutiles:
    del globals()[metrics['Stock'][i]]
    nombres_archivos_csv.remove(metrics['Stock'][i])

# Eliminar los stocks inútiles del DataFrame "Metrics"
metrics = metrics.drop(archivos_inutiles)"""

In [108]:
"""# Reiniciar los indices de metrics para evitar problemas futuros 
metrics.reset_index(drop=True, inplace=True)
metrics.index += 0"""

In [109]:
"""# Convertir toda la fila en int
metrics['Dividendos'] = pd.to_numeric(metrics['Dividendos'])"""

* Retention Ratio = (Net Income - Dividens)/Net Income

Hay tres Stocks que aún no tienen net income en el último trimestre, para ellos se ingresó la información del trimestre anterior de forma manual

In [42]:
"""for i in range(len(metrics['Stock'])):
    if metrics['Net Income'][i] == 0:
        print(metrics['Stock'][i])"""

In [ ]:
"""for i in range(len(metrics['Stock'])):
    if metrics['Stock'][i] == 'EMN':
        metrics['Net Income'][i] = 134000000
    if metrics['Stock'][i] == 'IDXG':
        metrics['Net Income'][i] = 430000
    if metrics['Stock'][i] == 'PERI':
        metrics['Net Income'][i] = 23790000"""

In [43]:
"""RR = []
for i in range(len(metrics['Stock'])):
    RR.append((metrics['Net Income'][i] - metrics['Dividendos'][i])/(metrics['Net Income'][i]))
metrics['Retentio Ratio'] = RR"""

Terminal Growth Rate (g)
1. Sustainable Growth Rate

* ROE

In [9]:
# WEB SCRAPING PARA EXTRAER EL ROE (Return On Equity)
def get_roe(ticker_symbol):
    url = f"https://www.marketwatch.com/investing/stock/{ticker_symbol}/company-profile?mod=mw_quote_tab"

    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Buscar el elemento que contiene el ROE
        roe_element = soup.find("td", string="Return on Equity")

        if roe_element:
            # Obtener el valor del ROE del siguiente elemento td en la misma fila
            roe_value = roe_element.find_next("td").text.strip()
            return roe_value
        else:
            return "ROE no encontrado en la página."

    except requests.exceptions.HTTPError:
        return "Error al realizar la solicitud. Verifica el ticker symbol o inténtalo más tarde."

In [80]:
"""#roe = []
for i in range(2100, len(metrics['Stock'])):
    roe.append(get_roe(metrics['Stock'][i]))
# metrics['ROE (%))'] = roe"""

In [83]:
"""no_access = 0
no_info = 0
for i in range(len(roe)):
    if roe[i] == 'Error al realizar la solicitud. Verifica el ticker symbol o inténtalo más tarde.':
        no_access += 1
    if roe[i] == 'ROE no encontrado en la página.':
        no_info += 1
print('Sin acceso:',no_access,'/','Sin info:',no_info)"""

Sin acceso: 0 / Sin info: 21


In [ ]:
"""# Para hacer el data cleaning todos los datos de roe deben ser str
for i in range(len(metrics)):
    roe[i] = str(roe[i])

# Continuar con el data cleaning
for i in range(len(roe)):
    roe[i] = roe[i].replace('%','')
    roe[i] = roe[i].replace(',','')
metrics['ROE (%)'] = roe"""

Algunos stocks no tienen ROE, esto puede ser porque hay falta de datos financieros de la empresa, la empresa está en una etapa muy temprana, porque hay problemas contables, o porque la empresa es privada. En ninguna de estas situaciones es recomendable confiar en una empresa que no brinda información para calcular el ROE

In [16]:
"""archivos_inutiles = []
for i in range(len(metrics['Stock'])):
    if metrics['ROE (%)'][i] == 'ROE no encontrado en la página.':
        archivos_inutiles.append(i)
    if metrics['ROE (%)'][i] == 'nan':
        archivos_inutiles.append(i)
        
# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for i in archivos_inutiles:
    del globals()[metrics['Stock'][i]]
    nombres_archivos_csv.remove(metrics['Stock'][i])

# Eliminar los stocks inútiles del DataFrame "Metrics"
metrics = metrics.drop(archivos_inutiles)"""

In [17]:
"""# Reiniciar los indices de metrics para evitar problemas futuros 
metrics.reset_index(drop=True, inplace=True)
metrics.index += 0"""

In [18]:
"""# Convertir toda la fila en número
metrics['ROE (%)'] = pd.to_numeric(metrics['ROE (%)'])"""

* Sustainable Growth Rate (g) = ROE * Retention Ratio

In [74]:
"""gr = []
for i in range(len(metrics['Stock'])):
    gr.append(metrics['ROE (%)'][i] * metrics['Retentio Ratio'][i])
metrics['Sustainable Growth Rate'] = gr"""

Historical Growth Rate = ((CV/PV)^(1/N)) - 1 
* N = 5 -> ese es el número de trimestres que tenemos en el histórico

In [ ]:
# Hay tres stocks que no tienen el net income del primer trimestre, se hace el ajuste manual
for i in range(len(metrics['Net Income (Pri. Tri)'])):
    if metrics['Net Income (Pri. Tri)'][i] == 0:
        print(metrics['Stock'][i])

No tiene sentido que se saque una raiz de un número negativo, es por eso que se hace el paralelismo con el segundo metodo en el for, para aproximarse al crecimiento de la empresa

In [328]:
N = 5
hgr = []
for i in range(len(metrics['Stock'])):  
    if  metrics['Net Income (Ult. Tri)'][i] > 0 and  metrics['Net Income (Pri. Tri)'][i] > 0:
        hgr.append((((metrics['Net Income (Ult. Tri)'][i] / metrics['Net Income (Pri. Tri)'][i])**(1/N))-1)*100)
    if metrics['Net Income (Ult. Tri)'][i] < 0 or  metrics['Net Income (Pri. Tri)'][i] < 0:
        hgr.append(((metrics['Net Income (Ult. Tri)'][i] - metrics['Net Income (Pri. Tri)'][i]) / (metrics['Net Income (Pri. Tri)'][i]) / 5)*100)
        
# metrics['Historical Growth Rate (%)'] = hgr

C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_14160\1815060931.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  hgr.append(((metrics['Net Income (Ult. Tri)'][i] - metrics['Net Income (Pri. Tri)'][i]) / (metrics['Net Income (Pri. Tri)'][i]) / 5)*100)


In [326]:
hgr

[1.9650357939162033,
 -23.715846994535518,
 -23.715846994535518,
 15.305391391876078,
 -21.134223054503643,
 -0.6891617497268299,
 2.2093219787654217,
 -2.330998581962762,
 17.328914331846,
 -39.74126807062651,
 -7.006130914768505,
 -23.715846994535518,
 -35.86583142097041,
 -0.5015298056424089,
 -23.715846994535518,
 1.239387071986453,
 1.2949219181020988,
 -7.245154620843985,
 -23.715846994535518,
 -23.715846994535518,
 -23.715846994535518,
 -55.94500435136147,
 -38.18087601306077,
 -23.715846994535518,
 -23.715846994535518,
 -35.156845748950595,
 19.236458526312262,
 2.3454654145628417,
 1.9824005459434302,
 -23.715846994535518,
 -23.715846994535518,
 -23.715846994535518,
 -23.715846994535518,
 -23.715846994535518,
 -35.67018995563648,
 158.44613782065787,
 -23.715846994535518,
 4.533446816694386,
 -5.652236486756812,
 -23.715846994535518,
 4.428511003502167,
 1.9752089918270155,
 -23.715846994535518,
 8.404464551525837,
 -10.196534071336727,
 -23.715846994535518,
 1.081698136816755

In [105]:
# del metrics['Historical Growth Rate (%)']

## METRICS

In [10]:
# Leer el aarchivo
# metrics = pd.read_csv('C:\\Users\\Leidy Hernandez\\Desktop\\Soy Henry\\Proyecto Integrador\\HENRY-Proyecto-Integrador\\Metrics')

In [332]:
# Guardar el archivo
# metrics.to_csv('Metrics',index=False)